The purpose of this notebook is for two things.

1. Explore and examine the TSV file containing the data of locations
2. Implementation of edit distance computation using dynamic programming

In [24]:
import csv
import math

In [25]:
# Incrase field size limit for this tsv file. Fixes some weird bug
csv.field_size_limit(1000000)

131072

In [212]:
# Covert tsv file to csv file
# Practice reading value from tsv and csv
# NOT NECESSARY
with open('cities_canada-usa.tsv','r',encoding='UTF-8') as tsvin, open('new.csv', 'w') as csvout:
    tsvin = csv.reader(tsvin, delimiter='\t')
    csvout = csv.writer(csvout) 
    for index, row in enumerate(tsvin):
        csvout.writerow(row)

In [233]:
def edge_d(a, b):
    '''
    Compute the edit distance between two words using dynamic programming
    Input: Word a, Word b
    Output: Graph of distances. A 2-dimensional array
    '''
    
    # Create graph nxm
    n = len(a)
    m = len(b)
    graph = []
    
    # Edge Case
    if n == 0:
        return m
    elif m == 0:
        return n
    
    # One extra column / row for initial empty state
    n += 1
    m += 1
    
    for i in range(n):
        graph.append([0] * m)
    
    # Call helper with graph
    return helper(a, b, graph)

def helper(a, b, graph):
    
    # Solve smallest subproblem 
    n = len(a) + 1
    m = len(b) + 1
    
    for i in range(n):
        graph[i][0] = i
    
    for i in range(1, m):
        graph[0][i] = i
        
    # Solve subproblems recursively
    for i in range(1, n):
        for j in range(1, m):
            diff = 1 if a[i-1] != b[j-1] else 0
            graph[i][j] = min(graph[i-1][j] + 1, graph[i][j-1] + 1, graph[i-1][j-1] + diff)
    return graph
    
edge_d('exponential', 'polynomial')

# edge_d('Calgary', '')
# edge_d('Snow', 'Know')

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [2, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [3, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10],
 [4, 3, 2, 3, 4, 5, 5, 6, 7, 8, 9],
 [5, 4, 3, 3, 4, 4, 5, 6, 7, 8, 9],
 [6, 5, 4, 4, 4, 5, 5, 6, 7, 8, 9],
 [7, 6, 5, 5, 5, 4, 5, 6, 7, 8, 9],
 [8, 7, 6, 6, 6, 5, 5, 6, 7, 8, 9],
 [9, 8, 7, 7, 7, 6, 6, 6, 6, 7, 8],
 [10, 9, 8, 8, 8, 7, 7, 7, 7, 6, 7],
 [11, 10, 9, 8, 9, 8, 8, 8, 8, 7, 6]]

## Create a table called cities

In [117]:
import sys
import psycopg2
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

create_table_command = (
    """
    CREATE TABLE cities (
        id TEXT PRIMARY KEY,
        name VARCHAR(200),
        ascii VARCHAR(200),
        alt_name VARCHAR(5000),
        lat FLOAT,
        long FLOAT,
        feature_class VARCHAR(1),
        feature_code VARCHAR(10),
        country_code VARCHAR(2),
        alt_country_code VARCHAR(60),
        admin_1_code VARCHAR(20),
        admin_2_code VARCHAR(20),
        admin_3_code VARCHAR(20),
        admin_4_code VARCHAR(20),
        population BIGINT,
        elevation INTEGER,
        dem INTEGER,
        timezone VARCHAR(40),
        modification_date VARCHAR(10)
    )
    """
)

con = None
con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')
try:
    cur = con.cursor()
    cur.execute(create_table_command)
    cur.close()
    con.commit()
    print('Successfully created table')
except (Exception, psycopg2.DatabaseError) as error:
    print(error)


In [216]:
# Populate database from CSV File
con = None
con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')
cur = con.cursor()

with open('cities_canada-usa.tsv','r',encoding='UTF-8') as tsvin:
    next(tsvin)
    cur.copy_from(tsvin, 'cities', sep='\t', null='')
    
con.commit()

In [205]:
# Delete table cities
con = None
con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')
cur = con.cursor()

try:
    cur.execute('DELETE FROM cities')
except (Exception, psycopg2.DatabaseError) as error:
    print(f'Error: {error}')
finally:
    con.commit()
    cur.close()
    con.close()

Error: relation "cities" does not exist
LINE 1: DELETE FROM cities
                    ^



In [285]:
# Create class of data objects

class City:
    def __init__(self, name, lat, long):
        self.name = name
        self.lat = lat
        self.long = long
        self.confidence_level = 0.0
        self.edit_distance = 0
        self.physical_distance = None
        
    def __repr__(self):
        return f'{self.name}  (Confidence {self.confidence_level}) (Edit Distance {self.edit_distance}) (Distance {self.physical_distance})'

In [339]:
# Read all values from postgres database
THRESHOLD = 3

def search(query, lat=None, long=None):
    con = None
    con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')
    cur = con.cursor()
    cur.execute('SELECT * FROM cities')
    rows = cur.fetchall()
    
    similar_cities = []
    for row in rows:
        confidence_graph = edge_d(query, row[1])
        if confidence_graph[-1][-1] < THRESHOLD:
            city = City(row[1], row[4], row[5])
            city.edit_distance = confidence_graph[-1][-1]
            if lat and long:
                city.physical_distance = lat_long_d(lat, long, row[4], row[5])
            
            calc_confidence_level(city)
            similar_cities.append(city)
            

    cur.close()
    con.close()
    return similar_cities
    
search('Toronot')

[Toronto  (Confidence 0.7142857142857143) (Edit Distance 2) (Distance 0.0),
 Toronto  (Confidence 0.7142857142857143) (Edit Distance 2) (Distance 0.0)]

In [342]:
results = search('Yoronto', 41.43979137, -88.1234974)
sorted_search = sorted(results, key=lambda x: x.confidence_level, reverse=True)
sorted_search

[Toronto  (Confidence 0.7542638647393582) (Edit Distance 1) (Distance 640.9839627813359),
 Toronto  (Confidence 0.7506422156163697) (Edit Distance 1) (Distance 755.8167686816727)]

In [244]:
def lat_long_d(_lat_a, _long_a, _lat_b, _long_b):
    '''
    Returns distance between two points in km
    '''
    lat_a = math.radians(_lat_a)
    lat_b = math.radians(_lat_b)
    long_a = math.radians(_long_a)
    long_b = math.radians(_long_b)
    
    # approximate radius of earth in km
    R = 6373.0
    dlon = long_b - long_a
    dlat = lat_b - lat_at

    a = math.sin(dlat / 2)**2 + math.cos(lat_a) * math.cos(lat_b) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance
lat_long_d(45.5017089, -73.5728828, 43.4821971, -80.6167601)

602.0879382136474

In [337]:
def calc_confidence_level(city):
    '''
    Input: City object variable
    Calculates and stores the confidence level of the city in-place (city variable is updated)
    
    Confidence level is calculated as follows:
    
    -\-\-\-\-\-\
    
    Output: Calculate confidence level
    '''
    
    INERTIA = 10
    confidence_level = 0.0
    if city.physical_distance:
        confidence_level = 0.8 * (INERTIA / (INERTIA + city.edit_distance**2)) + 0.2 * ( 1 / (1 + city.physical_distance / 100))
    else:
        confidence_level = (INERTIA / (INERTIA + city.edit_distance**2))
    city.confidence_level = confidence_level
    return city.confidence_level
    